# Part 1

1. mlflow server --host 127.0.0.1 --port 8080


In [1]:
import mlflow

mlflow.set_tracking_uri(uri="http://localhost:8080")

In [ ]:
import mlflow
from mlflow.models import infer_signature

import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)


In [ ]:
# Set our tracking server uri for logging
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

# Create a new MLflow Experiment
mlflow.set_experiment("MLflow Quickstart")

# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart",
    )


In [ ]:
# Load the model back for predictions as a generic Python Function model
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

predictions = loaded_model.predict(X_test)

iris_feature_names = datasets.load_iris().feature_names

result = pd.DataFrame(X_test, columns=iris_feature_names)
result["actual_class"] = y_test
result["predicted_class"] = predictions

result[:4]


# Part 2

In [14]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature


In [15]:
# Load dataset
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)

# Split the data into training, validation, and test sets
train, test = train_test_split(data, test_size=0.25, random_state=42)
train_x = train.drop(["quality"], axis=1).values
train_y = train[["quality"]].values.ravel()
test_x = test.drop(["quality"], axis=1).values
test_y = test[["quality"]].values.ravel()
train_x, valid_x, train_y, valid_y = train_test_split(
    train_x, train_y, test_size=0.2, random_state=42
)
signature = infer_signature(train_x, train_y)


In [16]:
def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
    # Define model architecture
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)
    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(1),
        ]
    )

    # Compile model
    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=params["lr"], momentum=params["momentum"]
        ),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    # Train model with MLflow tracking
    with mlflow.start_run(nested=True):
        model.fit(
            train_x,
            train_y,
            validation_data=(valid_x, valid_y),
            epochs=epochs,
            batch_size=64,
        )
        # Evaluate the model
        eval_result = model.evaluate(valid_x, valid_y, batch_size=64)
        eval_rmse = eval_result[1]

        # Log parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        # Log model
        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}


In [17]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result


In [18]:
space = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0),
}


In [19]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=8,
        trials=trials,
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")


2024/10/02 23:21:41 INFO mlflow.tracking.fluent: Experiment with name '/wine-quality' does not exist. Creating a new experiment.


Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 31s 698ms/step - loss: 34.3123 - root_mean_squared_error: 5.8577
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 7.0641 - root_mean_squared_error: 2.5088 - val_loss: 0.5832 - val_root_mean_squared_error: 0.7636

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.5883 - root_mean_squared_error: 0.7670
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5881 - root_mean_squared_error: 0.7666 - val_loss: 0.5187 - val_root_mean_squared_error: 0.7202

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.7323 - root_mean_squared_error: 0.8557
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5256 - root_mean_squared_error: 0.7244 - val_loss: 0.5492 - val_root_mean_squared_error: 0.7411

 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.5170 - root_mean_squared_error: 0.7191
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

2024/10/02 23:21:53 INFO mlflow.tracking._tracking_service.client: 🏃 View run classy-fox-329 at: http://127.0.0.1:8080/#/experiments/477308564409235171/runs/4b99bc79974d4dc6a769362d4598e8f7.

2024/10/02 23:21:53 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/477308564409235171.



Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 22s 496ms/step - loss: 40.3450 - root_mean_squared_error: 6.3518
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 38.9961 - root_mean_squared_error: 6.2443 - val_loss: 35.8787 - val_root_mean_squared_error: 5.9899

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 35.8345 - root_mean_squared_error: 5.9862
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 34.3594 - root_mean_squared_error: 5.8614 - val_loss: 32.0145 - val_root_mean_squared_error: 5.6581

Epoch 3/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 32.3662 - root_mean_squared_error: 5.6891
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 30.7383 - root_mean_squared_error: 5.5440 - val_loss: 28.5865 - val_root_mean_squared_error: 5.3466

 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step -

2024/10/02 23:22:01 INFO mlflow.tracking._tracking_service.client: 🏃 View run rambunctious-fly-280 at: http://127.0.0.1:8080/#/experiments/477308564409235171/runs/226d5b6e147b4231bb76eeb13d76d6aa.

2024/10/02 23:22:01 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/477308564409235171.



...

Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 21s 479ms/step - loss: 28.5678 - root_mean_squared_error: 5.3449
17/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 30.4934 - root_mean_squared_error: 5.5216   
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 30.6281 - root_mean_squared_error: 5.5341 - val_loss: 29.2294 - val_root_mean_squared_error: 5.4064

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 31.0661 - root_mean_squared_error: 5.5737
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 28.9188 - root_mean_squared_error: 5.3773 - val_loss: 27.1673 - val_root_mean_squared_error: 5.2122

Epoch 3/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 25.9542 - root_mean_squared_error: 5.0945
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 26.7074 - root_mean_squared_error: 5.1678 - val_lo

2024/10/02 23:22:48 INFO mlflow.tracking._tracking_service.client: 🏃 View run able-fawn-369 at: http://127.0.0.1:8080/#/experiments/477308564409235171/runs/74e8752dff1c490dbc0e65f82781dc08.

2024/10/02 23:22:48 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/477308564409235171.



100%|██████████| 8/8 [01:06<00:00,  8.28s/trial, best loss: 0.7410722970962524]


2024/10/02 23:22:54 INFO mlflow.tracking._tracking_service.client: 🏃 View run aged-toad-505 at: http://127.0.0.1:8080/#/experiments/477308564409235171/runs/dcf0f237709a48d581816193077c32a6.
2024/10/02 23:22:54 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/477308564409235171.


Best parameters: {'lr': 0.05009041204671813, 'momentum': 0.8551367227388531}
Best eval rmse: 0.7410722970962524
